In [22]:
import os
import copy
import torch
from tqdm import tqdm
import pandas as pd
import detectron2
from detectron2.data import detection_utils as utils
from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.data.datasets import register_coco_instances
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader

In [23]:
# Register Dataset
try:
    register_coco_instances('coco_trash_test', {}, '../dataset/test.json', '../dataset/')
except AssertionError:
    pass

In [24]:
# config 불러오기
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file('COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml'))

In [25]:
# config 수정하기
cfg.DATASETS.TEST = ('coco_trash_test',)

cfg.DATALOADER.NUM_WOREKRS = 2

cfg.OUTPUT_DIR = './output'

cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, 'model_final.pth')

cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 10
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.3

In [26]:
# model
predictor = DefaultPredictor(cfg)

# mapper - input data를 어떤 형식으로 return할지
def MyMapper(dataset_dict):
    
    dataset_dict = copy.deepcopy(dataset_dict)
    image = utils.read_image(dataset_dict['file_name'], format='BGR')
    dataset_dict['image'] = image
    
    return dataset_dict

# test loader
test_loader = build_detection_test_loader(cfg, 'coco_trash_train', MyMapper)

[10/01 08:27:23 d2.data.datasets.coco]: Loaded 4883 images in COCO format from ../dataset/train.json
[10/01 08:27:23 d2.data.build]: Distribution of instances among all 10 categories:
|   category    | #instances   |  category   | #instances   |  category  | #instances   |
|:-------------:|:-------------|:-----------:|:-------------|:----------:|:-------------|
| General trash | 3966         |    Paper    | 6352         | Paper pack | 897          |
|     Metal     | 936          |    Glass    | 982          |  Plastic   | 2943         |
|   Styrofoam   | 1263         | Plastic bag | 5178         |  Battery   | 159          |
|   Clothing    | 468          |             |              |            |              |
|     total     | 23144        |             |              |            |              |
[10/01 08:27:23 d2.data.common]: Serializing 4883 elements to byte tensors and concatenating them all ...
[10/01 08:27:23 d2.data.common]: Serialized dataset takes 2.17 MiB


In [27]:
# output 뽑은 후 sumbmission 양식에 맞게 후처리
prediction_strings = []
file_names = []

class_num = 10

for data in tqdm(test_loader):
    prediction_string = ''
    data = data[0]
    outputs = predictor(data['image'])['instances']
    
    targets = outputs.pred_classes.cpu().tolist()
    boxes = [i.cpu().detach().numpy() for i in outputs.pred_boxes]
    scores = outputs.scores.cpu().tolist()
    
    for target, box, score in zip(targets,boxes,scores):
        prediction_string += (str(target) + ' ' + str(score) + ' ' + str(box[0]) + ' ' 
        + str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' ')
        
    prediction_strings.append(prediction_string)
    file_names.append(data['file_name'].replace('../dataset/',''))

submission = pd.DataFrame()
submission['PredictionString'] = prediction_strings
submission['image_id'] = file_names
submission.to_csv(os.path.join(cfg.OUTPUT_DIR, f'submission_det2.csv'), index=None)
submission.head()

100%|██████████| 4883/4883 [05:53<00:00, 13.81it/s]


,PredictionString,image_id
0,0 0.8832188844680786 201.25818 196.56818 748.7...,train/0000.jpg
1,4 0.839802622795105 721.321 302.91956 994.1416...,train/0001.jpg
2,3 0.9941684007644653 271.5992 161.31232 902.21...,train/0002.jpg
3,0 0.7065123319625854 765.49866 2.3723438 968.2...,train/0003.jpg
4,1 0.963070273399353 468.479 429.58984 766.0945...,train/0004.jpg
